# Let's Install the Requirements

In [ ]:
!pip install layer --upgrade -q
!pip install sentencepiece -q
!pip install transformers -q

     |████████████████████████████████| 447 kB 5.3 MB/s 
     |████████████████████████████████| 271 kB 47.1 MB/s 
     |████████████████████████████████| 1.3 MB 41.3 MB/s 
     |████████████████████████████████| 4.4 MB 44.9 MB/s 
     |████████████████████████████████| 26.7 MB 49.9 MB/s 
     |████████████████████████████████| 159 kB 50.3 MB/s 
     |████████████████████████████████| 101 kB 7.2 MB/s 
     |████████████████████████████████| 56 kB 2.2 MB/s 
     |████████████████████████████████| 381 kB 47.8 MB/s 
     |████████████████████████████████| 4.0 MB 36.5 MB/s 
     |████████████████████████████████| 40 kB 4.9 MB/s 
     |████████████████████████████████| 16.8 MB 23.5 MB/s 
     |████████████████████████████████| 4.0 MB 35.8 MB/s 
     |████████████████████████████████| 256 kB 47.8 MB/s 
     |████████████████████████████████| 132 kB 47.4 MB/s 
     |████████████████████████████████| 212 kB 49.2 MB/s 
     |████████████████████████████████| 2.4 MB 35.9 MB/s 
     |████████████

In [ ]:
from layer.decorators import dataset, model,resources, pip_requirements, fabric
from layer import Model
import layer
import torch
import random
import re
import pandas as pd
import numpy as np
import os

## Getting started with Layer:
- Login to Layer Console
- Initialize the project

In [ ]:
# Login to Layer:
layer.login()

Please open the following link in your web browser. Once logged in, copy the code and paste it here.
https://app.layer.ai/oauth/authorize?response_type=code&code_challenge=cHP7ajj_JrNpJPp__t2q3e0ddu1BW6f4PBx46HG9qss&code_challenge_method=S256&client_id=0STDdcnpK48P8A429EAAn93WNuLmViLR&redirect_uri=https://app.layer.ai/oauth/code&scope=offline_access&audience=https://app.layer.ai
Code: YEfx0q49OMLrEVwNcCgIvPnboe5l8WukkePNgNbXoacqF
Successfully logged into https://app.layer.ai


In [ ]:
# Initialize the project:
layer.init('GPT2_text_summarization')

Project(name='GPT2_text_summarization', raw_datasets=[], derived_datasets=[], models=[], path=PosixPath('.'), project_files_hash='', readme='', account=Account(id=UUID('c9442765-3fe4-4729-a7e4-d3fc9b07726b'), name='aymane_hachcham'), _id=UUID('f29c025a-0d92-4913-94ff-035d4de50df7'), functions=[])

## Process and Load the Dataset to Layer

In [ ]:
# Mount Google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Load the dataset:
data_path = '/content/drive/MyDrive/GPT2/Dataset/Reviews.csv'
import pandas as pd

reviews = pd.read_csv(data_path)

In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [ ]:
reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


## Data Processing and Cleaning

In [ ]:
# Check for null values:
print(reviews.isna().sum())

# Remove null values:
reviews.dropna(inplace=True)

# Combining the two columns review and summary:
reviews['training'] = reviews['Text']  + 'TL;DR' + reviews['Summary']

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64


In [ ]:
reviews[:1500].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 0 to 1499
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Id                      1500 non-null   int64 
 1   ProductId               1500 non-null   object
 2   UserId                  1500 non-null   object
 3   ProfileName             1500 non-null   object
 4   HelpfulnessNumerator    1500 non-null   int64 
 5   HelpfulnessDenominator  1500 non-null   int64 
 6   Score                   1500 non-null   int64 
 7   Time                    1500 non-null   int64 
 8   Summary                 1500 non-null   object
 9   Text                    1500 non-null   object
 10  training                1500 non-null   object
dtypes: int64(5), object(6)
memory usage: 140.6+ KB


In [ ]:
# We take 5000 rows from the whole data. So can avoid GPU out of memory:
reviews = reviews[['Summary','Text','training']][:5000]

In [ ]:
# Save the dataset to Layer:
@dataset('amazon_reviews')
@resources(data_path)
def read_reviews():
    df = pd.read_csv(data_path) 
    # Remove null values:
    df.dropna(inplace=True)  
    # Combining the two columns review and summary:
    df['training'] = df['Text'] + 'TL;DR' + df['Summary'] 
    df = df[['Summary','Text','training']][:5000]
    return df

read_reviews()

In [ ]:
# Take the average length of the reviews:
sum_all_tokens = sum([len(review.split()) for review in reviews['training']])
avg_length = sum_all_tokens / len(reviews['training'])
avg_length

78.1608

Since the average instance length in words is 80, we can assume that a max length of 150 will cover most of the instances.

In [ ]:
max_length = 100

In [ ]:
reviews['training']

0       I have bought several of the Vitality canned d...
1       Product arrived labeled as Jumbo Salted Peanut...
2       This is a confection that has been around a fe...
3       If you are looking for the secret ingredient i...
4       Great taffy at a great price.  There was a wid...
                              ...                        
4995    I really wanted to like these.<br /><br />Firs...
4996    I was not impressed with these cookies when I ...
4997    The cookies came sealed and seem to be high qu...
4998    These taste very good, but aren't like the BES...
4999    I love these cookies. I am on the paleo diet r...
Name: training, Length: 5000, dtype: object

In [ ]:
@model("gpt2-tokenizer")
@fabric("f-medium")
def build_tokenizer():
    from transformers import AutoTokenizer
    # Load tokenizer from Hugging face
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    return tokenizer
  
layer.run([build_tokenizer])

⠋  gpt2-tokenizer       ━━━━━━━━━╸ TRAINING [0:00:13]                                   
   ↳ ]8;id=737965;https://app.layer.ai/aymane_hachcham/GPT2_text_summarization/models/gpt2-tokenizer\https://app.layer.ai/aymane_hachcham/GPT2_text_summarization/models/gpt2-tokenizer]8;;\

Run(project_name='GPT2_text_summarization')

## Load the Model and the Tokenizer from Layer

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
 
class GPT2ReviewDataset(Dataset):  
    def __init__(self, tokenizer, reviews, max_len):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.eos = self.tokenizer.eos_token
        self.eos_id = self.tokenizer.eos_token_id
        self.reviews = reviews
        self.result = []

        for review in self.reviews:
            # Encode the text using tokenizer.encode(). We add EOS at the end
            tokenized = self.tokenizer.encode(review + self.eos)
            
            # Padding/truncating the encoded sequence to max_len 
            padded = self.pad_truncate(tokenized)            

            # Creating a tensor and adding to the result
            self.result.append(torch.tensor(padded))

    def __len__(self):
        return len(self.result)


    def __getitem__(self, item):
        return self.result[item]

    def pad_truncate(self, name):
        extra_length = 4
        name_length = len(name) - extra_length
        if name_length < self.max_len:
            difference = self.max_len - name_length
            result = name + [self.eos_id] * difference
        elif name_length > self.max_len:
            result = name[:self.max_len + 3]+[self.eos_id] 
        else:
            result = name
        return result

In [ ]:
tokenizer = layer.get_model('aymane_hachcham/GPT2_text_summarization/models/tokenizer:2.2').get_train()

In [ ]:
# Fetching the data from Layer to format the Dataset object:
reviews = layer.get_dataset('layer/california_housing/datasets/train:1.1').to_pandas()

In [ ]:
reviews_dataset = GPT2ReviewDataset(tokenizer, reviews['trainig'], max_length)

Token indices sequence length is longer than the specified maximum sequence length for this model (1430 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
# Example of the review_dataset:
for item in reviews_dataset: 
  print(tokenizer.decode(item)), print(item)
  break

I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.TL;DRGood Quality Dog Food<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>
tensor([   40,   423,  5839,  1811,   286,   262, 28476,   414, 32530,  3290,
         2057,  3186,   290,   423,  1043,   606,   477,   284,   307,   286,
        

In [ ]:
dataloader = DataLoader(reviews_dataset, batch_size=32, shuffle=True)

In [ ]:
for s, n in enumerate(dataloader):
  print(s, n)
  break

0 tensor([[49123, 17166,  9155,  ..., 50256, 50256, 50256],
        [ 3666,   807, 27406,  ..., 50256, 50256, 50256],
        [   40,   373, 12451,  ..., 50256, 50256, 50256],
        ...,
        [44614, 12882,  8887,  ...,  2048,   355, 50256],
        [ 4863,   718,  1227,  ..., 11676,    11, 50256],
        [   40,   423, 15203,  ..., 50256, 50256, 50256]])


In [ ]:
def train(model, optimizer, dl, epochs):    
    for epoch in range(epochs):
        for idx, batch in enumerate(dl):
             with torch.set_grad_enabled(True):
                optimizer.zero_grad()
                batch = batch.to('cuda')
                output = model(batch, labels=batch)
                loss = output[0]
                loss.backward()
                optimizer.step()
                if idx % 50 == 0:
                    print("loss: %f, %d"%(loss, idx))

In [ ]:
@model("gpt2_text_summarization", dependencies=[Model("gpt2-tokenizer")])
@fabric("f-gpu-small")
@pip_requirements(packages=["torch","transformers","sentencepiece"])
def build_model():
    from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
    from torch import cuda
    import torch
    from torch.utils.data import Dataset, DataLoader
    from transformers import AutoModelWithLMHead
    
    parameters={
        "BATCH_SIZE":32,          
        "EPOCHS":10,              
        "LEARNING_RATE":3e-4,          
        "MAX_TARGET_TEXT_LENGTH":max_length
    }

    # Log parameters to Layer
    layer.log(parameters)

    # Get the tokenizer:
    tokenizer = layer.get_model('aymane_hachcham/GPT2_text_summarization/models/tokenizer:2.2').get_train()
  
    # Load pretrained model from Hugging face
    model = AutoModelWithLMHead.from_pretrained("gpt2")
    device = 'cuda' if cuda.is_available() else 'cpu'
    model.to(device)

    optimizer = torch.optim.AdamW(params = model.parameters(), lr=parameters['LEARNING_RATE'])

    train(model, optimizer, dataloader, epochs=parameters['EPOCHS'])
    
    return model

In [ ]:
layer.run([build_model], debug=True)

✅  gpt2_text_summariza… ━━━━━━━━━━ DONE [0:44:23]                                           
    ↳ ]8;id=108052;https://app.layer.ai/aymane_hachcham/GPT2_text_summarization/models/gpt2_text_summarization?v=3.2\https://app.layer.ai/aymane_hachcham/GPT2_text_summarization/models/gpt2_text_summariz]8;;\ 
    ]8;id=108052;https://app.layer.ai/aymane_hachcham/GPT2_text_summarization/models/gpt2_text_summarization?v=3.2\ation?v=3.2]8;;\

✅  gpt2_text_summariza… ━━━━━━━━━━ DONE [0:44:23]                                           
    ↳ ]8;id=238998;https://app.layer.ai/aymane_hachcham/GPT2_text_summarization/models/gpt2_text_summarization?v=3.2\https://app.layer.ai/aymane_hachcham/GPT2_text_summarization/models/gpt2_text_summariz]8;;\ 
    ]8;id=238998;https://app.layer.ai/aymane_hachcham/GPT2_text_summarization/models/gpt2_text_summarization?v=3.2\ation?v=3.2]8;;\

✅  gpt2_text_summariza… ━━━━━━━━━━ DONE [0:44:23]                                           
    ↳ ]8;id=638600;https://app.layer.ai/aymane_hachcham/GPT2_text_summarization/models/gpt2_text_summarization?v=3.2\https://app.layer.ai/aymane_hachcham/GPT2_text_summarization/models/gpt2_text_summariz]8;;\ 
    ]8;id=638600;https://app.layer.ai/aymane_hachcham/GPT2_text_summarization/models/gpt2_text_summarization?v=3.2\ation?v=3.2]8;;\

Run(project_name='GPT2_text_summarization')

In [ ]:
import torch

def topk(probs, n=9):
    # The scores are initially softmaxed to convert to probabilities
    probs = torch.softmax(probs, dim= -1)
    
    # PyTorch has its own topk method, which we use here
    tokensProb, topIx = torch.topk(probs, k=n)
    
    # The new selection pool (9 choices) is normalized
    tokensProb = tokensProb / torch.sum(tokensProb)

    # Send to CPU for numpy handling
    tokensProb = tokensProb.cpu().detach().numpy()

    # Make a random choice from the pool based on the new prob distribution
    choice = np.random.choice(n, 1, p = tokensProb)
    tokenId = topIx[choice][0]

    return int(tokenId)

In [ ]:
def model_infer(model, tokenizer, review, max_length=15):
    # Preprocess the init token (task designator)
    review_encoded = tokenizer.encode(review)
    result = review_encoded
    initial_input = torch.tensor(review_encoded).unsqueeze(0).to('cpu')

    with torch.set_grad_enabled(False):
        # Feed the init token to the model
        output = model(initial_input)

        # Flatten the logits at the final time step
        logits = output.logits

        # Make a top-k choice and append to the result
        result.append(topk(logits))

        # For max_length times:
        for _ in range(max_length):
            # Feed the current sequence to the model and make a choice
            input = torch.tensor(result).unsqueeze(0).to('cpu')
            output = model(input)
            logits = output.logits
            res_id = topk(logits)

            # If the chosen token is EOS, return the result
            if res_id == tokenizer.eos_token_id:
                return tokenizer.decode(result)
            else: # Append to the sequence 
                result.append(res_id)

    # IF no EOS is generated, return after the max_len
    return tokenizer.decode(result)

In [ ]:
samples = [review.split('TL;DR')[0] for review in list(reviews['training'].sample(n=3, random_state=1)['training'])]

In [ ]:
gtp2_model = layer.get_model('aymane_hachcham/GPT2_text_summarization/models/gpt2_text_summarization:3.1').get_train()

In [ ]:
# Then you can output the summary prediction from the samples using model_infer.

for review in samples:
    summaries = set()
    print(review)
    while len(summaries) < 3:
        summary = model_infer(model, tokenizer, review + " TL;DR ").split(" TL;DR ")[1].strip()
        if summary not in summaries:
            summaries.add(summary)
    print("Summaries: "+ str(summaries) +"\n")